## Example of 3D Graph

Here we provide the example code for SphereNet on QM93D and MD17 datasets. You can easily replace SphereNet with SchNet and DimeNetPP by changing model name and model parameters.

In [53]:
import torch
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../..')
from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.method import SphereNet #SchNet, DimeNetPP
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator
from torch_geometric.utils import degree

In [49]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)


### Example code for MD17 data

We trained a separate model for each molecule. You can use default hyperparameters to get comparable results, we also tuned hyperparameters like lr, lr_decay_factor, lr_decay_step_size, batch_size, basis_emb_size_dist, basis_emb_size_angle, basis_emb_size_torsion to achieve better performance. The values/search space for hyperparameters are listed in the Appendix of our paper.

The default hyperparameters for MD17 are:  
    &ensp; energy_and_force=True, cutoff=5.0, num_layers=4, hidden_channels=128, out_channels=1, int_emb_size=64,  
    &ensp; basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256,  
    &ensp; num_spherical=7, num_radial=6, envelope_exponent=5,  
    &ensp; num_before_skip=1, num_after_skip=2, num_output_layers=3,  
    &ensp; epochs=2000, batch_size=4, vt_batch_size=64, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=200.

#### Loading dataset

In [ ]:
dataset_md17 = MD17(root='dataset/', name='aspirin')

split_idx_md17 = dataset_md17.get_idx_split(len(dataset_md17.data.y), train_size=1000, valid_size=10000, seed=42)

train_dataset_md17, valid_dataset_md17, test_dataset_md17 = dataset_md17[split_idx_md17['train']], dataset_md17[split_idx_md17['valid']], dataset_md17[split_idx_md17['test']]
print('train, validaion, test:', len(train_dataset_md17), len(valid_dataset_md17), len(test_dataset_md17))

#### Loading model, loss and evaluation function

We predict energy and take the negative of the derivative of the energy with respect to the atomic positions as predicted forces.

The evaluation metric is mean absolute error (MAE).

In [ ]:
model_md17 = SphereNet(energy_and_force=True, cutoff_i=10.0, cutoff_e=5.0, num_layers=4, 
        hidden_channels=128//2, out_channels=1, int_emb_size_T=64//2, int_emb_size_Q=32//2, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256//2,
        num_bilinear_T=64//2, num_bilinear_Q=32//2,
        num_spherical=7, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3, fix=True, use_bilinear=True
        )
loss_func_md17 = torch.nn.L1Loss()
evaluation_md17 = ThreeDEvaluator()

#### Training

In [ ]:
run3d_md17 = run()
run3d_md17.run(device, train_dataset_md17, valid_dataset_md17, test_dataset_md17, model_md17, loss_func_md17, evaluation_md17, epochs=5, batch_size=4, vt_batch_size=64, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=200, energy_and_force=True)